# Calculate recovery of clinvar variants in each refseq region
#### Pavlos Bousounis, 30 Oct 2019
##### Updated 11/04/2019

In [5]:
import os
import numpy as np
import pandas as pd

In [4]:
os.getcwd()

'/Users/pbousounis/Experiments/2019-10-29_hg19mod/2019-10-29-RefSeq-CV_recovery'

### Import ClinVar bed file

In [70]:
cv_bed_colnames = ['chr', 'start', 'end', 'name']
cv_bed_file = '/Users/pbousounis/Experiments/2019-10-29_hg19mod/2019-11-06_refseq_cv_slop_intersect/bed/2019-11-05_clinvar_path-l.bed'
cv_bed = pd.read_csv(cv_bed_file, sep='\t', header=None, names=cv_bed_colnames)

cv_bed.head()

,chr,start,end,name
0,chr1,949522,949523,183381_ISG15
1,chr1,949695,949696,161455_ISG15
2,chr1,949738,949739,161454_ISG15
3,chr1,957604,957605,243036_AGRN
4,chr1,957692,957693,243037_AGRN


### Get number of ClinVar variants by gene

In [71]:
# extract gene names from 'name' into new column
cv_bed['gene'] = cv_bed['name'].str.extract(r'(\d+_)(\w+)')[1]
cv_bed.head()

,chr,start,end,name,gene
0,chr1,949522,949523,183381_ISG15,ISG15
1,chr1,949695,949696,161455_ISG15,ISG15
2,chr1,949738,949739,161454_ISG15,ISG15
3,chr1,957604,957605,243036_AGRN,AGRN
4,chr1,957692,957693,243037_AGRN,AGRN


In [72]:
cv_bed['total_gene_vars'] = cv_bed.groupby('gene')['gene'].transform('count')
cv_bed.head()

,chr,start,end,name,gene,total_gene_vars
0,chr1,949522,949523,183381_ISG15,ISG15,3.0
1,chr1,949695,949696,161455_ISG15,ISG15,3.0
2,chr1,949738,949739,161454_ISG15,ISG15,3.0
3,chr1,957604,957605,243036_AGRN,AGRN,15.0
4,chr1,957692,957693,243037_AGRN,AGRN,15.0


In [73]:
# save the modified table
cv_bed.to_csv(cv_bed_file, sep='\t', index=False, header=True)

### Import RefSeq exons table

In [16]:
rs_exons_file = '/Users/pbousounis/Experiments/2019-10-29_hg19mod/2019-10-29-RefSeq-CV_recovery/2019-11-04_RefSeq_parsed_bed.bed'
rs_exons_colnames = ['chr', 'start', 'end', 'strand', 'name']
rs_exons = pd.read_csv(rs_exons_file, sep='\t', header=None, low_memory=False, names=rs_exons_colnames)
rs_exons.head()

,chr,start,end,strand,name
0,1,11873,12226,+,NR_046018
1,1,12612,12720,+,NR_046018
2,1,13220,14408,+,NR_046018
3,1,29320,29369,-,NR_024540
4,1,24737,24890,-,NR_024540


In [17]:
# # remove 'chr' prefix from chromosome IDs column
# rs_exons[0] = rs_exons[0].str.extract('(chr)(\w+)')[1]
# rs_exons[0].astype('int64').dtypes
# rs_exons.head()

## Define function: ***bed_not()***

In [18]:
import pybedtools
from pybedtools import BedTool


# define the not-element-of function to get regions unique to the test_bed file
def bed_not(ref_bed_filepath, test_bed_filepath):
    
    """ Given two bed files, ref_bed and test_bed, perform bedtools intersect -v to return only
    regions in the ref_bed file that do NOT overlap any regions in the test_bed file."""
    
    cwd = os.getcwd()
    
    # specify the reference bed file
    ref_bedtool = BedTool(ref_bed_filepath)
    prfx_ref = ref_bed_filepath.split('/')[-1]
    prfx_ref = prfx_ref.split('.')[0]
    
    # specify the new ClinVar bed file
    test_bedtool = BedTool(test_bed_filepath)
    prfx_test = test_bed_filepath.split('/')[-1]
    prfx_test = prfx_test.split('.')[0]

    # specify name/path of output bed file
    out_bed_filepath = '{}/{}_NotIn_{}.bed'.format(cwd, prfx_test, prfx_ref)
    
    # run bedtools intersect to get all test_bed regions NOT found in ref_bed (-v option)
    ref_NotIn_test = test_bedtool.intersect(b=ref_bedtool, v=True)
    ref_NotIn_test.head()
    
    ref_NotIn_test.saveas(out_bed_filepath, trackline="track name='ClinVar loci NOT IN TSO bed file'")
    feat_count = ref_NotIn_test.count()
    
    # confirm file saved
    print('\nNumber of {} features NOT IN {} = {}'.format(prfx_test, prfx_ref, feat_count))
    
    if os.path.exists(out_bed_filepath):
        print('Success! File saved to {}.'.format(os.path.join(cwd, out_bed_filepath)))

    return(feat_count)

In [19]:
x = bed_not(cv_bed_file, rs_exons_file)

1	11873	12226	+	NR_046018
 1	12612	12720	+	NR_046018
 1	13220	14408	+	NR_046018
 1	29320	29369	-	NR_024540
 1	24737	24890	-	NR_024540
 1	18267	18365	-	NR_024540
 1	17914	18060	-	NR_024540
 1	17605	17741	-	NR_024540
 1	17232	17367	-	NR_024540
 1	16857	17054	-	NR_024540
 
Number of 2019-11-04_RefSeq_parsed_bed features NOT IN clinvar_path-l = 517734
Success! File saved to /Users/pbousounis/Experiments/2019-10-29_hg19mod/2019-10-29-RefSeq-CV_recovery/2019-11-04_RefSeq_parsed_bed_NotIn_clinvar_path-l.bed.
